In [1]:
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris, load_digits

from ipywidgets import interact

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
np.random.seed(1024)

In [3]:
# dataset = load_digits()
dataset = load_iris()
X, y = dataset.data, dataset.target
# X = X / 255
X = StandardScaler().fit_transform(X)

In [4]:
@interact(method=['auto', 'full', 'arpack', 'randomized'])
def do_pca(method):
    pca = PCA(n_components=2, svd_solver=method, random_state=1024)
    print(pca)
    X2d = pca.fit_transform(X)
    plt.scatter(X2d[:,0], X2d[:,1], c=y)
    plt.matshow(pca.components_, cmap='viridis')
    plt.show()

interactive(children=(Dropdown(description='method', options=('auto', 'full', 'arpack', 'randomized'), value='…

In [5]:
from bqplot import *
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector,
)
from ipywidgets import Output, ToggleButtons, VBox, HTML
from ipywidgets import interact, interactive_output


/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
out = Output(layout={'border': '1px solid black'})

In [7]:
def create_figure(x_data, y_data, color, selector, **selector_kwargs):
    '''
    Returns a Figure with a Scatter and a Selector.
    
    Arguments
    ---------
    selector: The type of Selector, one of
        {'BrushIntervalSelector', 'BrushSelector', 'FastIntervalSelector', 'IndexSelector', 'LassoSelector'}
    selector_kwargs: Arguments to be passed to the Selector
    '''
    scales = {'x': LinearScale(), 'y': LinearScale()}
    
    scatter = Scatter(x=x_data, y=y_data, scales=scales, color=color,
                      selected_style={'opacity': '1'}, unselected_style={'opacity': '0.2'})
    scatter.enable_move=True
    sel = selector(marks=[scatter], **selector_kwargs)
    
#     text_brush = HTML()
#     if selector != LassoSelector:
#         def update_text(*args):
#             text_brush.value = '{}.selected = {}'.format(selector.__name__, sel.selected)
#         sel.observe(update_text, 'selected')
#         update_text()

    x_ax = Axis(scale=scales['x'])
    x_ay = Axis(scale=scales['y'], orientation='vertical')
    fig = Figure(marks=[scatter], axes=[x_ax, x_ay], title='{} Example'.format(selector.__name__),
                 interaction=sel)
    return VBox([fig, text_brush])

In [8]:
pca = PCA(n_components=2, svd_solver='auto', random_state=1024)
X2d = pca.fit_transform(X)
# create_figure(x_data=X2d[:,0], y_data=X2d[:,1], color=y, selector=LassoSelector)

In [23]:
scales = {'x': LinearScale(), 'y': LinearScale(), 'color': OrdinalColorScale(scheme='CATEGORY10')}
scatter = Scatter(x=X2d[:, 0], y=X2d[:, 1], scales=scales, color=y,
                  selected_style={'opacity': '1'}, unselected_style={'opacity': '0.2'})
scatter.enable_move=True
sel = LassoSelector(marks=[scatter])

#     text_brush = HTML()
#     if selector != LassoSelector:
#         def update_text(*args):
#             text_brush.value = '{}.selected = {}'.format(selector.__name__, sel.selected)
#         sel.observe(update_text, 'selected')
#         update_text()

x_ax = Axis(scale=scales['x'])
x_ay = Axis(scale=scales['y'], orientation='vertical')
fig = Figure(marks=[scatter], axes=[x_ax, x_ay],
             interaction=sel)
VBox([fig,out])

In [24]:
fig.interaction=None
scatter.enable_move=True

In [31]:
out.clear_output()

In [26]:
scatter.selected=list(range(95,105))

In [ ]:
sel.marks=[scatter]
fig.interaction=sel

In [13]:
target_id = None
old_pos = None

In [14]:
@out.capture()
def on_moving_started(source_obj, target):
    global target_id
    global old_pos
    pos = target['point']
    old_pos = [pos['x'], pos['y']]
    target_id = target['index']

In [15]:
scatter.on_drag_start(on_moving_started)

In [16]:
@out.capture()
def on_drag1(source_obj, target):
    scatter.x[0] = 1.0
    scatter.y[0] = -1.0
#     pos = target['point']
#     current_pos = [pos['x'], pos['y']]
#     dx = current_pos[0] - old_pos[0]
#     dy = current_pos[1] - old_pos[1]
    
#     old_x = scatter.x
#     old_y = scatter.y
#     print(dx, dy)
#     for pid in scatter.selected:
#         if pid != target['index']:
#             old_x[pid] += dx
#             old_y[pid] += dy
#     scatter.x = old_x
#     scatter.y = old_y
    
scatter.on_drag(on_drag1)
scatter.hold_sync()

In [29]:
@out.capture()
def on_moving_finished(source_obj, target):
    pos = target['point']
    current_pos = [pos['x'], pos['y']]
    dx = current_pos[0] - old_pos[0]
    dy = current_pos[1] - old_pos[1]
    
    print('end move ', target['index'])
    for pid in scatter.selected:
        if pid != target['index']:
            print('update pos: ', pid)
            scatter.x[pid] += dx
            scatter.y[pid] += dy

In [30]:
scatter.on_drag_end(on_moving_finished)